In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv(
    Path('Resources/lending_data.csv')
)

# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df['loan_status']

# Separate the X variable, the features
X = df.drop(columns="loan_status")

In [4]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_test, X_train, y_test, y_train = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Predictions": predictions, "Actual": y_test}).head()

,Predictions,Actual
44997,0,0
13468,0,0
52163,0,0
5330,0,0
9982,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9538324075554845

In [11]:
# Generate a confusion matrix for the model
matrix = confusion_matrix(y_test, predictions)
print(matrix)

[[55973   304]
 [  163  1712]]


In [12]:
# Print the classification report for the model
risk = ["Healthy Loan", "High Risk Loan"]
print(classification_report(y_test, predictions, target_names=risk))

                precision    recall  f1-score   support

  Healthy Loan       1.00      0.99      1.00     56277
High Risk Loan       0.85      0.91      0.88      1875

      accuracy                           0.99     58152
     macro avg       0.92      0.95      0.94     58152
  weighted avg       0.99      0.99      0.99     58152



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The accuracy score from the classification report is very high at 99%. We have healthy loans that are predicted with extremely high accuracy, at 100% and 99% for precision and recall respectively. High risk loans also performed quite well at 85% for precision and 91% for recall. 

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [13]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
resampled_data = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_train_resampled, y_train_resampled = resampled_data.fit_resample(X_train, y_train)

In [14]:
# Count the distinct values of the resampled labels data
y_train_resampled.value_counts()

loan_status
0    18759
1    18759
Name: count, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [15]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resampled_model = LogisticRegression(random_state=1)
# Fit the model using the resampled training data
resampled_model.fit(X_train_resampled, y_train_resampled)

# Make a prediction using the testing data
resampled_pred = resampled_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [19]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_resampled = balanced_accuracy_score(y_test, resampled_pred)
balanced_accuracy_resampled


0.9941258560335483

In [20]:
# Generate a confusion matrix for the model
resampled_matrix = confusion_matrix(y_test, resampled_pred)
print(resampled_matrix)

[[55946   331]
 [   11  1864]]


In [21]:
# Print the classification report for the model
resampled_risk = ["Healthy Loan", "High Risk Loan"]
print(classification_report(y_test, resampled_pred, target_names=resampled_risk))

                precision    recall  f1-score   support

  Healthy Loan       1.00      0.99      1.00     56277
High Risk Loan       0.85      0.99      0.92      1875

      accuracy                           0.99     58152
     macro avg       0.92      0.99      0.96     58152
  weighted avg       0.99      0.99      0.99     58152



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** With oversampling in this case, the Healthy Loan predictions `0` have the same score. However, there is a slight change with the High Risk Loan predictions where the recall score as changed from 91% to 99%, therefore increasing the balanced f1-score from a 88% to 92%. That is a 4% increase in prediction accuracy with High Risk loans. 